In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1e15')

# Load submissions
saspav_df = pd.read_csv('/home/code/exploration/datasets/santa-2025.csv')
jazivxt_df = pd.read_csv('/home/code/exploration/datasets/submission.csv')

print("Saspav submission shape:", saspav_df.shape)
print(saspav_df.head())
print("\nJazivxt submission shape:", jazivxt_df.shape)
print(jazivxt_df.head())